In [168]:
#pip install todoist-python
from todoist import TodoistAPI, models
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [169]:
credentials = dotenv_values('../../usage/.env')

In [170]:
api = TodoistAPI(credentials["TOKEN"])
api.sync()

{'collaborator_states': [],
 'collaborators': [],
 'day_orders': {},
 'day_orders_timestamp': '1649513087.7174382',
 'due_exceptions': [],
 'filters': [],
 'full_sync': False,
 'items': [],
 'labels': [],
 'live_notifications': [],
 'live_notifications_last_read_id': 2456784820,
 'notes': [],
 'project_notes': [],
 'projects': [],
 'reminders': [],
 'sections': [],
 'sync_token': 'XS4jFCEsW4Wu99JUhcQ7tqIdv0RZyCyZdZqWj-BrBaaXD4XK1jGLfPrLwFyMm9eRsdIT4mukOE8HPtT41OZzANhLN9SNMHKp22QKx39je89LkKk',
 'temp_id_mapping': {},
 'view_options': []}

In [171]:
def get_todoist_activities(api, pages, limit=100):
    activities = pd.DataFrame()
    for p in range(pages):
        if p > 0:
            activities = all_activities
        activities_on_page_df = pd.DataFrame(api.activity.get(page=p, limit=100)['events'])                                                                       
        all_activities = pd.concat([activities, activities_on_page_df])
    return all_activities

def get_parent_id(api, object_id):
    object_id = str(object_id)
    info_list = [api.items.get_by_id(object_id)] 
    for item in info_list:
        try:
            if ('item' in item.keys()) and ('parent_id' in item['item'].keys()):
                return item['item']['parent_id']
        except AttributeError:
            return None

In [172]:
df = get_todoist_activities(api, pages=30, limit=100)

In [173]:
df['content'] = df['extra_data'].apply(lambda x: x['content'] if 'content' in x.keys() else None)
df['due_date'] = df['extra_data'].apply(lambda x: x['due_date'] if 'due_date' in x.keys() else None)
df['name'] = df['extra_data'].apply(lambda x: x['name'] if 'name' in x.keys() else None)
object_id_list = [get_parent_id(api, idd) for idd in df['object_id'].unique()]
parent_id_dp = pd.DataFrame([object_id_list, df['object_id'].unique()]).T
parent_id_dp = parent_id_dp.rename(columns={0:'parent_id', 1: 'object_id'}).astype(int, errors='ignore')
df = df.merge(parent_id_dp, on='object_id')

In [174]:
df['event_date'].min(), df['event_date'].max()

('2021-09-26T13:08:43Z', '2022-04-09T16:07:55Z')

In [175]:
df.head(5)

,event_date,event_type,extra_data,id,initiator_id,object_id,object_type,parent_item_id,parent_project_id,content,due_date,name,parent_id
0,2022-04-09T16:07:55Z,completed,"{'client': 'Mozilla/5.0; Todoist/1990', 'conte...",15552099771,NaN,5751064441,item,NaN,2282993091,"Figura 5: deixar claro o que (-6,-6) é",None,None,5574607502
1,2022-04-09T16:02:16Z,added,"{'client': 'Mozilla/5.0; Todoist/1990', 'conte...",15552055514,NaN,5751064441,item,NaN,2282993091,"Figura 5: deixar claro o que (-6,-6) é",None,None,5574607502
2,2022-04-09T15:36:52Z,uncompleted,"{'client': 'Mozilla/5.0; Todoist/1990', 'conte...",15551859457,NaN,5646876550,item,NaN,2282993091,Deixar so o fit da parabola na figura 7,None,None,5574607502
3,2022-04-02T21:24:10Z,completed,"{'client': 'Mozilla/5.0; Todoist/1981', 'conte...",15463214960,NaN,5646876550,item,NaN,2282993091,Deixar so o fit da parabola na figura 7,None,None,5574607502
4,2022-03-04T00:48:19Z,added,"{'client': 'Mozilla/5.0; Todoist/1923', 'conte...",15089805396,NaN,5646876550,item,NaN,2282993091,Deixar so o fit da parabola na figura 7,None,None,5574607502


In [176]:
# df.to_csv('../../usage/data/raw/TODOIST-20210926-20220904.csv')